In [ ]:
import pandas as pd
from ftplib import FTP
from dna_puller.sliding_parser import SlidingParser
import os, gzip, json, csv, subprocess
from gnuplot_generator.gnuplot_generator import GnuplotGenerator

csv_folder = 'csv'
window_size = 1000
postfix = 'all'

In [ ]:
# This file is csv file from NCBI that contains name of species, other several atributtes and path of species data at FTP server 
# I recommend to create own CSV list on https://www.ncbi.nlm.nih.gov/genome/browse/#!/overview/
# I also using there Pandas library for more straightfoward navigation in CSV, but python's  csv reader works well too. 
df = pd.read_csv("species_NCBI.csv")

links = df[['GenBank FTP']].values
names = df[['#Organism Name']].values

In [ ]:
files = {}
paths = {}

def files_callback(name):
    files_array.append(name)

for index in range(len(links)):
    link = links[index][0]
    parts = link.split('/', maxsplit=3)
    
    next_path = (parts[3].split('/')[-1] + '_assembly_structure/') + 'Primary_Assembly/assembled_chromosomes/FASTA/'
    ftp = FTP(parts[2])
    ftp.login()
    ftp.cwd(parts[3] + '/' + next_path)
    paths[names[index][0]] = parts[3] + '/' + next_path
    files_array = []
    ftp.retrlines('NLST', callback = files_callback)
    files[names[index][0]] = files_array
    ftp.close()
    
for name, path in paths.items():
    server = 'ftp.ncbi.nlm.nih.gov'
    ftp = FTP(server)
    ftp.login()
    ftp.cwd(path)
    
    if not os.path.exists(name):
        os.mkdir(name)
    
    if not os.path.exists(name + '/' + 'dna_sm'):
        os.mkdir(name + '/' + 'dna_sm')
        
    for filename in files[name]:
        print('download file: ' + filename)
        file_path = os.path.join(name, 'dna_sm', filename)
        fasta_path = os.path.join(name, 'dna_sm', filename[0:-3])
        with open(file_path, 'wb') as file:
            ftp.retrbinary('RETR ' + filename, file.write)
        handle = gzip.open(file_path)
        with open(fasta_path, 'wb') as out:
            for line in handle:
                out.write(line)
    ftp.close()

In [ ]:
json_files = []

for name, in names:
    path = name + "/dna_sm"
    data = {'dna_sm': {}}
    
    files = os.listdir(path)
    fasta_files = []
    
    for file in files:
        if file.endswith(('.fa', '.fna')):
            fasta_files.append(file)
    
    for file in fasta_files:
        parser = SlidingParser(window_size)
        data['dna_sm'].update(parser.parse_file(path + "/" + file))
    
    if not os.path.exists('jsons'):
        os.mkdir('jsons')
        
    json_path = os.path.join('jsons' , name + '.json')
    print('creating ' + json_path)
    
    json_files.append(json_path)
    with open(json_path, 'w') as fp:
         json.dump(data, fp)

In [ ]:
for file in json_files:
    name, ext = os.path.splitext(os.path.basename(file))
    data = {}
    os.makedirs('csv/' + name, exist_ok=True)
    
    with open(file) as f:
        data = json.load(f)['dna_sm']
    
    for lg, values in data.items():
        if lg[:2] == 'LR' or lg[:2] == 'CM':
            print('lg', lg)
            with open('csv/' + name + '/' + lg + '.csv', 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile, delimiter=' ',
                                        quotechar='|', quoting=csv.QUOTE_MINIMAL)

                for index, value in values.items():
                    gc_count = value['G'] + value['C'] + value['S'] + value['g'] + value['c'] + value['s']
                    all_count = value['all'] - value['N'] - value['n']
                    gc_val = gc_count/all_count if all_count > 0 else 0

                    
                    big_percent = 0
                    if value['all_big'] > 0:  # XXX Important: all_big contains N, therefore is never zero
                        big_percent = value['all_big'] / float(value['all'])

                    csvwriter.writerow([index, gc_val, big_percent])

In [ ]:
def file_len(fname):
    with open(fname) as f:
        return sum(1 for _ in f)


In [ ]:
folders = os.listdir(csv_folder)
species_folders = [folder for folder in folders if not folder.startswith('.')]
species_folders

In [ ]:
files = {}
biggest_files = {}
sorted_files = {}
for folder in species_folders:
    biggest_files[folder] = 0
    sorted_files[folder] = {}
    csv_files = []
    for file in os.listdir(csv_folder + '/' + folder):
        if file[-4:] == '.csv':
            csv_files.append(file)
            
            len_f =  file_len(csv_folder + '/' + folder + '/' + file)
            sorted_files[folder][file] = len_f
            if biggest_files[folder] < len_f * 1000:
                biggest_files[folder] = len_f * 1000
    files[folder] = csv_files 
    sorted_files[folder] = sorted(sorted_files[folder], key = sorted_files[folder].get, reverse=True)

In [ ]:
def create_plot(fname):
    p = subprocess.Popen(['gnuplot', '-p', fname], stdout=subprocess.PIPE, 
                                                   stderr=subprocess.PIPE)
    result, err = p.communicate()
    if p.returncode != 0:
        raise IOError(err)       


for folder in species_folders:
    range_plot = '[0:' + str(biggest_files[folder]) + ']'
    gen = GnuplotGenerator(folder, range_plot)
    gen.add_palette("( 0 'green', 50 'orange', 100 'red')")
    print(f'Generating plot plots_{postfix}/{folder}_{postfix}.png')
    gen.set_term('png', 'plots_' + postfix + '/' + folder + '_' + postfix + '.png', 10000, 0.1)
    
    if not os.path.exists('plots_' + postfix):
        os.makedirs('plots_' + postfix)
    
    for file in sorted_files[folder]:
        gen.add_plot(csv_folder + '/' + folder + '/' + file, '1:2:3', file[:-4])
        break
    
    with open(csv_folder + '/' + folder + '/' + folder + '_' + postfix + '.gnu', 'w') as gnu_file:
        lines = gen.prepare_definition()
        for line in lines:
            gnu_file.write(line + '\n')
    
    create_plot(csv_folder + '/' + folder + '/' + folder + '_' + postfix + '.gnu')